<a href="https://colab.research.google.com/github/raggi-eth/Darkblock_Multiple_Forward_Looking/blob/main/Darkblock_UPP_uploader_Alpha_V0%2C5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ![darkblock](https://images.squarespace-cdn.com/content/v1/60709af07e296271b217ac52/e2e4c2b5-341d-4a4c-8668-9de83840e64b/White%403x.png?format=1500w)<br>
#@markdown <font color="white" size="+2">unlockble Swagbag generator</font></b><br>
#@markdown *turn your NFTs into a container full of encrypted content only accessible by the NFT owner

#@markdown https://www.darkblock.io

# @markdown ---
# @markdown <font color="white" size="+2"> multiple Darkblock uploader</font></b><br>
# @markdown **Mints multiple Darkblocks. The Darkblock should be minted prior to the NFT.<br>Upon minting a Darkblock you are returned a transaction id (tx_id)<br>that should be embedded into the NFT using the tag name "darkblock-id":(tx-id)<br><a href="https://api.opensea.io/api/v1/asset/0x495f947276749ce646f68ac8c248420045cb7b5e/30553606573219150352991292921105176340809048341686170040023897612118596255745">See example Ethereum NFT with Darkblock embedded</a><br>  <a href="https://api.all.art/v1/solana/4zY8xz9d1CHoxhz9NYa9qhfa4o67DYnS52H1vnBUz8MV">Example on Solana:</a><br>  Max file size is 215mb, anything larger will be rejected. Higher limits coming soon! Minting a large Darkblock can take several minutes. This is an asynchronous process. The data is uploaded, encrypted, and the Arweave transaction is calculated, but then to make the process quicker, an asynchronous process pushes the data to Arweave. It can take a few to several minutes for the Darkblock to be available, this is intentional to avoid frontrunning. To check the status of a Darkblock you can use the info endpoint.**


In [ ]:

#@markdown ## Helper Functions / Methods
#@markdown This code defines helper function to make life and debugging a little easier.
#@markdown ## Running the script with helper functions.
#@markdown **Fill out the form/variables then go to the Runtime menu and choose "Run All".**
#@markdown Alternatively, ensure your run the helper functions code, before any of the other code.

import os
import builtins  
#import __builtin__ 

# Defines Helper Functions

def create_folder(directory: str):
  """
  Checks if a directory exists and if not creates it.
  :param directory: Required folder and path (directory)
  """
  is_folder = os.path.isdir(directory)
  if not is_folder:
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Error creating folder: ", e)
            raise

    print("Successfully created folder : ", directory)
  else:
    print("Folder exists. Script will use existing folder at : ", directory)

def create_temporary_folders():
  """
  Creates some temporary directories.
  """
  tmpDir = ("/content/tmp")
  inputDir = ("/content/input")

  # If folder doesn't exist, then create it.
  create_folder(tmpDir)
  create_folder(inputDir)

def get_api_key() -> str:
  """
  Checks if API-Key has been entered previously and if so
  returns it.
  :return: api key as string
  """
  file = pathlib.Path("/content/tmp/apikey.json")
  if file.exists () == False:
    sys.exit("Enter API-key in the top cell")

  my_key = ''
  with open('/content/tmp/apikey.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    my_key = json_data["API_KEY"]
  return my_key


In [ ]:
#@markdown ---
#@markdown <font color="white" size="+2">Run this cell to enter your API KEY</font></b><br>
#@markdown minting Darkblocks requiers an API-key. Join the <a href="http://chat.darkblock.io/"> our discord </a> and request one
#@markdown  
API_KEY = 0
API_KEY = builtins.input("Enter API_KEY: ") 
#API_KEY  = input("enter API_KEY")

create_temporary_folders()

with open('/content/tmp/apikey.json', 'w') as f:
    f.write('{ \n')
    f.write('"API_KEY":"{}" \n'.format(API_KEY))
    f.write('}')
    f.close


#@markdown ---

apiKeyEntered = True


Enter API_KEY: nnv962zmngm3qxntt03rgx904pfj
Folder exists. Script will use existing folder at :  /content/tmp
Folder exists. Script will use existing folder at :  /content/input


# *

populate the input folder with the files you want to encrypt and access control by either

In [ ]:
#@markdown ---
#@markdown <font color="white" size="+2">choose the upload method and run cell </font></b><br>
upload_method = 'mount_google_drive' #@param ["browser_upload", "mount_google_drive"]
path_to_drive_folder = "UPP/testBeans" #@param {type:"string"}

#@markdown ---
import os

create_temporary_folders()

if upload_method == "browser_upload":
    from google.colab import files
    %cd /content/input
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    input_folder="/content/input"

else:
    from google.colab import drive
    drive.mount('/content/drive')
    input_folder="/content/drive/MyDrive/{}".format(path_to_drive_folder)
    print ("/content/drive/MyDrive/{} is your input folder".format(path_to_drive_folder))
    IDcacheFolder = "/content/drive/MyDrive/{}/tx-idCache/".format(path_to_drive_folder)
    
    create_folder(IDcacheFolder)  

Folder exists. Script will use existing folder at :  /content/tmp
Folder exists. Script will use existing folder at :  /content/input
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/UPP/testBeans is your input folder
Folder exists. Script will use existing folder at :  /content/drive/MyDrive/UPP/testBeans/tx-idCache/


In [ ]:


# @markdown ---
# @markdown ## Basic settings:


# @markdown ---
# @markdown *enter the wallet address that will deploy the minting contract<br> *for solana the creator needs the be verified
creator_address = ""# @param {type:"string"}

nft_platform = 'Polygon'# @param ['Solana', 'Tezos', 'Polygon', 'Avalanche', 'Ethereum']
nft_standard = 'ERC-1155'# @param ['ERC-721', 'ERC-1155', 'Metaplex']
# @markdown ---
# @markdown ## optional settings:
darkblock_name = "" # @param {type:"string"}
darkblock_description = "" # @param {type:"string"}
publisher_address = ""  # @param {type:"string"}
# @markdown ---
# @markdown ## NFT metadata settings:

generate_metadata = True  # @param {type:"boolean"}
collection_size =   None# @param {type:"integer"}
NFT_name = ""  # @param {type:"string"}
NFT_description = ""  # @param {type:"string"}
external_link = ""# @param {type:"string"}
image_uri = ""# @param {type:"string"}
file_extension = ""# @param {type:"string"}
url_to_multiple_image_files = "" # @param {type:"string"}
# @markdown *url to a folder with images with incremented names; 1.png, 2.png... <br> *on Solana the name scheme starts with 0.png
same_image_for_all = False  # @param {type:"boolean"}
url_to_single_image = ""# @param {type:"string"}

# @markdown ---
# @markdown ## solana specific settings:

#creators_address = "7C2jmV8Za4y8uYWbXQsKb3cURC41yvVMG8FWaTW6seTe" # @param {type:"string"}
royaltyShare = "" # @param {type:"string"}







              

In [ ]:
import os
import requests
import numpy as np
import shutil
import pathlib
import json
import glob
import errno
import sys


# @markdown ## ← run to generate Darkblocks + metadata


#path to content to spawn darkblocks
dir_path = r'{}'.format(input_folder)

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
res.sort()

params = {
    'apikey': '{}'.format(get_api_key()),
}
filenames = []
response_list = []
for filename in res:
    print(filename)
    files = {
        'file': open('{}/%s'.format(input_folder) % filename, 'rb'),
        'creator_address': (None, '{}'.format(creator_address)),
        'nft_platform': (None, '{}'.format(nft_platform)),
        'nft_standard': (None, '{}'.format(nft_standard)),
        'darkblock_description': (None, '{}'.format(NFT_description)),
        'publisher_address': (None, '{}'.format(publisher_address)),
        'Name': (None, '{}'.format(filename)),
    }
    filenames.append(filename)
    response = requests.post(
        'https://api.darkblock.io/v1/darkblock/mint', params=params, files=files)
    response_list.append(response.text.encode('utf8'))


new_array = np.array(response_list)
print(new_array)

with open("/content/tmp/tmp.txt", "w+") as f:
    data = f.read()
    f.write(str(new_array))

f1 = open("/content/tmp/tmp.txt", "r+")
input = f1.read()
input = input.replace(',', '\n')
f2 = open("/content/tmp/tmp2.txt", "w+")
f2.write(input)
f1.close()
f2.close()

tx_id = []
value = []

f = open("/content/tmp/tmp2.txt")
with open("/content/tmp/tmp2.txt") as f:
    for line in f:
        if line.startswith('"tx_id"'):
            value.append("darkblock-id")
            tx_id.append(line[:-1])



darkblock_id = [s.replace('"tx_id":', '') for s in tx_id]
darkblock_id = [s.replace('"', '') for s in darkblock_id]
print(darkblock_id)

if input_folder == '/content/input':
  metadataDir = ("/content/metadata")
else:
  metadataDir = ("/content/drive/MyDrive/{}/metadata".format(path_to_drive_folder))

# If folder doesn't exist, then create it.
create_folder(metadataDir)

if generate_metadata == True:
################
# EVM METADATA #
################
  if nft_platform == "Polygon" or nft_platform == "Avalanche" or nft_platform == "Ethereum":
      for x in darkblock_id:
        extension = (filename[max(filename.find('.'),0):].strip())
        with open("/content/tmp/meta.txt", "w+") as f:
            f.write('  "traits": [')

        with open("/content/tmp/meta.txt", "a") as f:
          for x in darkblock_id:  
            f.write('{'+'"trait_type": "darkblock-id", "value": "{}"'.format(x)+'},')

        with open("/content/tmp/meta.txt", 'rb+') as filehandle:
            filehandle.seek(-1, os.SEEK_END)
            filehandle.truncate()
            
        with open("/content/tmp/meta.txt", "a") as f:
            f.write(']\n}')

        
      for itt in range(0, collection_size):
        token_id = itt+1
        print(token_id)
        print("{} meta-sippi".format(token_id))
        
        if same_image_for_all == True:
          with open("/content/tmp/meta2.txt", "w") as f:
            f.write("{")
            f.write('\n  "description": "{}",\n  "image": "{}",\n  "name": "{}-#{}",\n  "external_link": "{}",\n'.format(NFT_description, url_to_single_image, NFT_name, token_id, external_link))
      
        else:
          with open("/content/tmp/meta2.txt", "w") as f:
            f.write("{")
            f.write('\n  "description": "{}",\n  "image": "{}{}.{}",\n  "name": "{}-#{}",\n  "external_link": "{}",\n'.format(NFT_description, url_to_multiple_image_files, token_id, file_extension ,NFT_name, token_id, external_link))

        #apend traits to rest of metadata
        firstfile = ("/content/tmp/meta2.txt")
        secondfile = ("/content/tmp/meta.txt")
        f1 = open(firstfile, 'r')
        f2 = open(secondfile, 'r')
        f1.close()
        f2.close()
        f1 = open(firstfile, 'a+')
        f2 = open(secondfile, 'r')        
        f1.write(f2.read())
        f1.seek(0)
        f2.seek(0)
        f1.close()
        f2.close()

        #save as json
        if input_folder == '/content/input':
          shutil.copy("/content/tmp/meta2.txt", "/content/metadata/{}.json".format(token_id))
        else:
          shutil.copy("/content/tmp/meta2.txt", "/{}/{}.json".format(metadataDir, token_id))
################
# SOL METADATA #
################
  else:
    openDict = "{"
    closeDict = "}"
    extension = (filename[max(filename.find('.'),0):].strip())
    with open("/content/tmp/metasol.txt", "w+") as f:
      f.write('"properties":{"files":[')
      for x in darkblock_id: 
        f.write('{}"uri":"{}","type":"encrypted({})"{},'.format(openDict, x, extension, closeDict))
        f.close
    with open("/content/tmp/metasol.txt", 'rb+') as f:
        f.seek(-1, os.SEEK_END)
        f.truncate()
        f.close
    with open("/content/tmp/metasol.txt", "a") as f:    
        f.write("]"+"," +'"creators":[{}"address":"{}","share":{}{}]{}'.format(openDict, creator_address, royaltyShare, closeDict, closeDict))
        f.close

    for itt in range(collection_size, 1):
            print("{} meta-sippi (solana)".format(itt))
            
            if same_image_for_all == True:
              with open("/content/tmp/metasol2.txt", "w") as f:
                f.write("{")
                f.write('\n  "description": "{}",\n  "image": "{}",\n  "name": "{}-#{}",\n  "external_link": "{}",\n  "traits": [\n'.format(NFT_description, image_uri, NFT_name, itt, external_link))
          
            else:
              with open("/content/tmp/metasol2.txt", "w") as f:
                f.write('\n  "description": "{}",\n  "image": "{}{}.{}",\n  "name": "{}-#{}",\n  "external_link": "{}",\n  "traits": [\n'.format(NFT_description, url_to_multiple_image_files, itt, file_extension ,NFT_name, itt, external_link))

            #apend traits to rest of metadata
            firstfile = ("/content/tmp/metasol2.txt")
            secondfile = ("/content/tmp/metasol.txt")
            f1 = open(firstfile, 'r')
            f2 = open(secondfile, 'r')
            f1.close()
            f2.close()
            f1 = open(firstfile, 'a+')
            f2 = open(secondfile, 'r')        
            f1.write(f2.read())
            f1.seek(0)
            f2.seek(0)
            f1.close()
            f2.close()

            if input_folder == '/content/input':
            #save as json and delete traits tmp file
              shutil.copy("/content/tmp/metasol2.txt", "/content/metadatasol/{}.json".format(token_id))
            else:
              shutil.copy("/content/tmp/metasol2.txt", "/{}/{}.json".format(metadataDir, token_id))

######################################
# darkblock id cache to google drive #
######################################
if input_folder == "/content/drive/MyDrive/{}".format(path_to_drive_folder):
    file = pathlib.Path("{}/darkblocks.txt".format(IDcacheFolder))
    if file.exists () == False:
        f1 = open(file, "w")
        f1.close
    
    print ("File exist")
    with open(file, "a") as f1:   
            for f, b in zip(filenames, darkblock_id):
                f1.write("file: " + f)
                f1.write("\ndarkblock-id: " + b)
                f1.write("\n")
                f1.write("-"*58)
                f1.write("\n")

1.png
2.png
[b'{"message":"Tx created successfully","tx_id":"uTUUmQne4TaWblPGfemv8yuroN6ClVo0l-omShKzl64","file_name":"1.png","file_size":202488,"request_duration":"1.927 seconds","gateways":[{"host":"arweave.net","protocol":"https","port":443}]}'
 b'{"message":"Tx created successfully","tx_id":"Hh1cQWzhnmNfhzDjo016Du-GV_h6m4sGJ72pOedHs_E","file_name":"2.png","file_size":202399,"request_duration":"1.807 seconds","gateways":[{"host":"arweave.net","protocol":"https","port":443}]}']
['uTUUmQne4TaWblPGfemv8yuroN6ClVo0l-omShKzl64', 'Hh1cQWzhnmNfhzDjo016Du-GV_h6m4sGJ72pOedHs_E']
/content/drive/MyDrive/UPP/testBeans/metadata folder already exists.
1 meta-sippi
2 meta-sippi
File exist


In [ ]:
#@markdown ---
#@markdown <font color="white" size="+2"> clear folder</font></b><br>

folder = 'input'# @param ['input', 'metadata']

import os, shutil
folder = '/content/{}'.format(folder)
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))